- <span style="color:blue">**Занятие 1 (NLP)**</span>
- <span style="color:gray">Занятие 2 (NLP)</span>
- <span style="color:gray">Занятие 3 (NLP)</span>
- <span style="color:gray">Занятие 4 (Recommender Systems)</span>
- <span style="color:gray">Занятие 5 (Recommender Systems)</span>
- <span style="color:gray">Занятие 6 (CRISP-DM)</span>
- <span style="color:gray">Занятие 7 (to be anounced)</span>
- <span style="color:gray">Занятие 8 (to be anounced)</span>
- <span style="color:gray">Занятие 9 (Class Hours)</span>
- <span style="color:gray">Занятие 10 (Class Hours)</span>

Тексты можно использовать в задачах машинного обучения точно так же, как обычные числовые данные. Единственное требование - нужно как-то представить их в виде числового вектора.

На этом занятии мы посмотрим, какие есть подходы для векторизации текстов и увидим, как эти подходы используются в контексте решения практических задач класификации.

#### Requirements
- sklearn
- nltk

#### Глоссарий
- <span style="color:blue">Document</span> - единица текста, соотвествующая одному наблюдению (статья, отзыв, предложение)
- <span style="color:blue">Corpus</span> - набор всех документов, которыми мы располагаем
- <span style="color:blue">Term</span> или <span style="color:blue">Token</span> - составная часть, на которую бъется документ (например, слово)
- <span style="color:blue">Document-Term</span> матрица - матрица, у которой по строкам отложены документы, а по столбцам термы, значением же является частота данного терма в документе (или другие характеристики). Также иногда работают с ее транспонриованным вариантом Term-Document matrix.
- <span style="color:blue">Vocabulary</span> - набор всех термов из корпуса и назначенных им порядковых номеров
- <span style="color:blue">Vector-Space Model</span> - способ представления терма в виде многомерного вектора
- <span style="color:blue">Term Frequency</span> - частота терма в данном документе
- <span style="color:blue">Document Frequency</span> - в каком проценте документов корпуса встречается данный терм
- <span style="color:blue">Inverse Docuemnt Frequency</span> - 1/DF, коэффициент "уникальности" терма

Создадим список из 3 документов

In [ ]:
text = ['To be, or not to be: that is the question:','to be be to', 'I went to party yesterday']
text

Попробуем его векторизовать. Для этого создаем объект класса **CountVectorizer** из библиотеки sklearn. В конструкторе пока используем все настройки по умолчанию.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

Класс CountVectorizer сождержит 2 классических для sklearn метода 
- fit() - построение словаря
- transform() - преобразование текста в соответствии с этим словарем

Поскольку часто эти действия выполняются одно за другим, есть еще метод
- fit_transform()

который выполняет их сразу оба. Этот метод обычно и используется

In [ ]:
text_vectorized = vect.fit_transform(text)

В результате вызова метода fit_transform заполняется словарь термов. Мы можем посмотреть, что в нем. 

In [ ]:
vect.vocabulary_

Видим, что каждому уникальному терму назначен свой порядковый номер (0,1,2...) (часто порядковые номера назначаются по мере того, как новые слова встречаются в документе, однако порядок нумерации здесь не имеет никакого значения, поэтому  подобное представление текста называется bag-of-words).

Кроме того на выходе мы получаем Document-Term матрицу (то есть набор векторизированных текстов)

In [ ]:
print(text_vectorized.shape)

Размерность получившейся document-term матрицы (3,11). Действительно, у нас 3 документа и можем посчитать, что в этих текстах 11 уникальных слов (местоимение "I" по умолчанию отфильтровывается как незначимое, так что оно не в счет).

In [ ]:
print(type(text_vectorized))

Видим, что на выходе у нас [sparse-матрица](https://en.wikipedia.org/wiki/Sparse_matrix). Такой формат позволяет эффективно хранить матрицы огромных размерностей, но из-за этого мы не можем отобразить ее на экране как обычную матрицу. Так как матрица небольшая, мы можем явно преобразовать её в dense-матрицу и вывести на экран.

In [ ]:
text_vectorized.todense()

Так как у нас **Count**Vectorizer, в результирующем векторе мы видим **число** вхождений кажого терма.

Как еще можно представлять текст.

Например, вместо слов часто используют 2-граммы. Для этого при создании векторайзера используем параметр ngram_range. Обратите внимание, что параметр должен быть типа Tuple из 2 элментов. Он задает интервал длин n-граммов, которые будут рассчитаны. В примере ниже считаются только 2-граммы.

In [ ]:
vect = CountVectorizer(ngram_range=(2,2))
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect.vocabulary_

Видим, что словарь выглядит уже по-другому - он состоит из всех уникальных последовательных пар слов, которые встречаются в тексте. 

Преобразованный с помощью этого словря текст представляет собой 3 вектора в пространстве большей размерности.

In [ ]:
text_vectorized.todense()

На практике часто комбинируют n-граммы разной размерности. При построении моделей это позволяет находить паттерны не только в словах, но и в словосочетаниях и устойчивых выражениях. 

Ниже пример, как рассчитать 1-граммы и 2-граммы.

In [ ]:
vect = CountVectorizer(ngram_range=(1,2))
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect.vocabulary_

In [ ]:
text_vectorized.todense()

По умолчанию тексты переводятся в нижний регистр. Иногда регистр важен, в этом случае, его можно отключить опцией lowercase.

In [ ]:
vect = CountVectorizer(lowercase=False)
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect.vocabulary_

Иногда с текстами работают не на уровне слов или n-грамов, а на уровне символов (это встречается, например, в задачах генерации). Мы можем задать, на что будем разбивать наши документы, параметром analyzer - для символов это analyzer='char', а для слов (по умолчанию) это analyzer='word'.

In [ ]:
vect = CountVectorizer(analyzer='char')
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect.vocabulary_

In [ ]:
text_vectorized.todense()

Также можем явно задать фильтрацию неинформативных стоп-слов. Они передаются списком.

In [ ]:
vect = CountVectorizer(stop_words=['to','a','then'])
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect.vocabulary_

Видим, что отфильтровалось слово to

In [ ]:
text_vectorized.todense()

Важная функция - мы можем отфильтровать термы со слишком большой или слишком малой частотой использования в корпусе (document frequency).

Большой DF говорит о том, что слово скорее всего малоинформативно - оно характерно для языка в целом, но не характерно для каких-либо классов документов (а это нехорошо, если мы захотим решать задачу классификации на этих данных). С большим DF обычно предоги, союзы и общеупотребительные слова и выражения.

Термы с малым DF слабо влияют на целевую переменную, занимая при этом место. Их можно спокойно отфильтровать.

Эти пороги задает пара параметров: min_df и max_df

In [ ]:
vect = CountVectorizer(min_df=0.01, max_df=0.5)
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect.vocabulary_

In [ ]:
text_vectorized.todense()

Если мы не хотим хранить и каждый раз обновлять словарь, можем воспользоваться алгоритмом хэширования слов. Аналогом класса CountVectorizer с реализованным hashing trick является класс **HashingVectorizer**. Определение индекса слова в векторном представлении документа просиходит динамически с помощью хэш-функции.

Обратите внимание, что по умолчанию, HasingVectorizer нормирует получаемый вектор документа. Для иллюстрации пока отключим нормировку.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

vect = HashingVectorizer(norm=None, alternate_sign=False)
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
vect

Обратите внимание на размерность данных

In [ ]:
text_vectorized.todense()

Ничего не видно. Чтобы найти индексы наших 8 термов можно сделать так:

In [ ]:
import numpy
numpy.nonzero(text_vectorized)

Индексы знаем, теперь можно посмотреть сами значения:

In [ ]:
text_vectorized[numpy.nonzero(text_vectorized)]

Еще один часто используемый векторайзер это класс **TfidfVectorizer**. Он является полным аналогом CountVectorizer, но по умолчанию включает TF-IDF преобразование частот.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(norm=False)
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
text_vectorized.todense()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(use_idf=False, norm='l2')
text_vectorized = vect.fit_transform(text)

print(text_vectorized.shape)
print(type(text_vectorized))
text_vectorized.todense()

Классы sklearn предоставляют базовый набор возможностей для предобработки текстовых данных. Когда нужно что-то более сложное, либо пишут обработчики самостоятельно, либо прибегают к использованию сторонних библиотек (например, NLTK).

Типичный набор преобразований текста выглядит так: 
- нормализация (удаление неинформативных символов, цифр)
- токенизация (разделение текста на смысловые единицы)
- лемматизация (приведение слова к нормальной форме, именительный падеж, единственное число)
- стемминг (выделение основы слова)
- выделение частей речи
- фильтрация стоп-слов (соединительных слов, союзов и предлогов, специфичных для доменной области неинформативных слов)

и так далее.

Далее несколько примеров преобразований из библиотеки NLTK

In [ ]:
from nltk import word_tokenize
doc = word_tokenize('Я пришел в столовую и съел волшебных котлет')
doc

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Only English
lemmatizer.lemmatize("fugitives")

In [ ]:
from nltk import pos_tag
tags = pos_tag(doc, lang='rus')
tags

Видим, что тэгер каждое слово текста относит к определенной части речи (V - глагол, CONJ - союз, PR - предлог и так далее). 

В основе тэгера лежит самообучаемая предиктивная модель. Она не использует словари, поэтому вполне работает и с выдуманными языком

In [ ]:
tags = pos_tag(word_tokenize('кареокий шляндр высвестил поорекомандно кнодлик'), lang='rus')
tags

Классы CountVectorizer, HashingVectorizer и TfidfVectorizer допускают кастомизацию. Например, мы можем реализовать свою логику обработки в специальном классе MyTokenizer.

Вся логика обработки реализуется в методе \__call\__, куда в качестве параметра передается документ для обработки (doc). Ниже пример такого класса из одного из проектов:

In [ ]:
class MyTokenizer(object):
    def __init__(self):
        self.wnl = Mystem()
    def __call__(self, doc):
        tokens = word_tokenize(doc)
        pos = pos_tag(tokens, lang='rus')
        pos = [x[0] for x in pos if (x[1] not in ["NONLEX","CONJ"]) or (x[0] == 'eos')]
        lemmatized_tokens = [self.wnl.lemmatize(t)[0] for t in pos]
        return (lemmatized_tokens)


Экземпляр этого класса передается в качестве параметра при создании векторайзера:

In [ ]:
vect = TfidfVectorizer(
    tokenizer = MyTokenizer(), 
    min_df=0.001,
    max_df=0.75, 
    stop_words = russian_stop_words, 
    lowercase=True, 
    ngram_range=(1,2))

text_vectorized = cv.fit_transform(text)

Попробуем решить реальную задачу. Для этого загрузим датасет с текстами с помощью встроенных средств sklearn.

In [5]:
from sklearn.datasets import fetch_20newsgroups
text_dataset = fetch_20newsgroups(subset='train', shuffle=True)

В датасете содержатся тексты 20 категорий. Их названия можно прочитать в переменной target_names

Нам нужно перевести тексты в векторное представление!

Повторяем то, что делали выше

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(text_dataset.data)
X_train_counts.shape

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

Для классификации будем использовать Multinomial NaiveBayes классификатор. Это простейший классификатор среди тех, которые обычно применяют для классификации текстовых данных. Он основан на подчсете частотности появления слов в каждом классе.

В метод fit() передаем обучающую выборку: матрицу предкиторов и целевую переменную

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, text_dataset.target)

Проверим как работает классификатор на 2 тестовых примерах

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
print(predicted)

In [3]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, text_dataset.target_names[category]))

NameError: name 'docs_new' is not defined

Последовательности трансформаций удобно объединять в пайплайны. Потом их можно обучать, так же как обычный классификатор.

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
])
text_clf.fit(text_dataset.data, text_dataset.target)

Делаем прогноз на тестовой части данных и проверяем его качество. Используем метрику Accuracy - показывает, в каком проценте тестовых кейсов угадываем правильный ответ.

In [14]:
# Load Test Dataset
text_dataset_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)
docs_test = text_dataset_test.data

# Apply Model to new data
predicted = text_clf.predict(docs_test)

# Assess Quality
import numpy as np
np.mean(predicted == text_dataset_test.target)

NameError: name 'text_clf' is not defined

Попробуем использовать алгоритм классификации SVM, который также является стандартным алгоритмом (эпохи доглубокого обучения) для работы с текстами.

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None))])

text_clf.fit(text_dataset.data, text_dataset.target)  
predicted = text_clf.predict(docs_test)
numpy.mean(predicted == text_dataset_test.target)            

Возможно сущесвтуеют параметры, используя которые, мы можем повысить точность модели? Узнать можно только одним способом - попробовать. Для этого сущесвтует такой механизм, как Grid Search. Это способ запуска обучения, при котором в цикле перебираются возможные наборы параметров, а затем находится оптмаильный.

Для начала необходимо создать сетку параметров, на которой мы будем искать оптимум.

По каждому параметру мы просто списком перечисляем значения, которые мы хотим, чтобы он принимал.

Можно задавать как непосредственно параметры моделей, так и параметры трансформаций. Для этого достаточно указать имя шага (как в пайплайне) и через два подчеркивания имя параметра.

In [ ]:
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
}

Для автоматизации процесса обучения на сетке параметров в sklearn существует класс GridSearchCV. Его параметры:
- Классификатор или пайплайн, который нужно запускать 
- Сетка параметров
- Вид кросс-валидации. Например, если cv=5, то будет использована 5-кратная k-fold валидация. Кросс-валидация нужна для получения более устойчивых метрик качества по каждому набору парметров. Значение метрики усредняется между этих 5 запусков.

In [ ]:
from sklearn.model_selection import GridSearchCV
cvGrid = GridSearchCV(text_clf, parameters, cv=5)

Созданная выше сетка предполагает 2^3 = 8 различных комбинаций параметров.

CV = 5 запускает процесс обучения 5 раз

Итого, требуется 5 * 8 = 40 запусков. Расчет может занять продолжительное время. Попробуем запуститься на небольшом помножестве выборки.

In [ ]:
cvGrid.fit(text_dataset.data[:400], text_dataset.target[:400])

Как посмотреть наилучшее значение метрики?

In [ ]:
cvGrid.best_score_

Есть словарик cv results_, который содержит всю информацию об обучении. Для нас наиболее важен массив mean_test_score:

In [ ]:
cvGrid.cv_results_['mean_test_score']

from Для сравнения попробуем обучаться всего на 4 отобранных категориях, как в документации. Это сократит объем обучающей выборки, и считаться будем быстрее. Кроме того, они мало пересекаются, поэтому можно ожидать, что точность прогноза будет еще выше.

In [17]:
from sklearn.datasets import fetch_20newsgroups
df = fetch_20newsgroups(subset="train")

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vectorized_text = vect.fit_transform(df['data'])


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('clf', SGDClassifier())])

text_clf.fit(df.data, df.target)


# docs_test = text_dataset_test.data
# predicted = text_clf.predict(docs_test)
# numpy.mean(predicted == text_dataset_test.target)            

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

In [16]:
vectorized_text

<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

Так и есть

Теперь повторим GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),
}
cvGrid = GridSearchCV(text_clf, parameters, cv=5)
cvGrid.fit(text_dataset.data, text_dataset.target)

In [ ]:
cvGrid.best_score_

Точность выше, чем на полном наборе данных.

Посмотрим полную информацию.

In [ ]:
cvGrid.cv_results_